### Bibliotecas

In [2]:
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

### Iniciando uma instância no Google Chrome

In [14]:
# A classe Service é usada para iniciar uma instância do Chrome WebDriver
service = Service()

# webdriver.ChromeOptions é usada para definir a preferência para o browser do Chrome
options = webdriver.ChromeOptions()

# Inicia-se a instância do Chrome WebDriver com as definidas 'options' e 'service'
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.gov.br/pf/pt-br/search?origem=form&SearchableText=licitacao"

driver.get(url)

### Encontrar Elementos do HTML
- find_element(By.ID, 'id')
- find_element(By.NAME, 'name')
- find_element(By.XPATH, 'xpath')
- find_element(By.LINK_TEXT, 'link text')
- find_element(By.PARTIAL_LINK_TEXT, 'partial link text')
- find_element(By.TAG_NAME, 'tag name')
- find_element(By.CLASS_NAME, 'class name')
- find_element(By.CSS_SELECTOR, 'css selector')

In [31]:
elementos = driver.find_elements(By.CSS_SELECTOR, 'span.titulo a')
print(elementos)

[<selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.454")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.455")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.456")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.457")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.458")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9c

In [33]:
len(elementos)

30

In [32]:
for elemento in elementos:
    print('Texto da notícia: ', elemento.text)
    print("Link: ", elemento.get_attribute('href'))
    print('##################################')

Texto da notícia:  PF e CGU deflagram operação de combate a crimes contra à administração pública e de lavagem de capitais em Pernambuco
Link:  https://www.gov.br/pf/pt-br/assuntos/noticias/2025/06/pf-e-cgu-deflagram-operacao-de-combate-a-crimes-contra-a-administracao-publica-e-de-lavagem-de-capitais-em-pernambuco
##################################
Texto da notícia:  PF desarticula esquema criminoso de desvio de verbas da saúde e da educação no interior do Piauí
Link:  https://www.gov.br/pf/pt-br/assuntos/noticias/2025/06/pf-desarticula-esquema-criminoso-de-desvio-de-verbas-da-saude-e-da-educacao-no-interior-do-piaui
##################################
Texto da notícia:  PF e PMSC prendem uruguaio procurado pela Interpol e apreendem droga, veículos e dinheiro em espécie
Link:  https://www.gov.br/pf/pt-br/assuntos/noticias/2025/05/pf-e-pmsc-prendem-cidadao-uruguaio-procurado-pela-interpol-e-apreende-droga-veiculos-e-dinheiro-em-especie
##################################
Texto da notícia:

In [ ]:
# //*[@id="search-results"]/div[4]/div[2]/span[2]/ul[2]/li[4]/a[1]

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

In [21]:

def iniciar_driver():
    service = Service()
    # webdriver.ChromeOptions é usada para definir a preferência para o browser do Chrome
    options = webdriver.ChromeOptions()

    # Inicia-se a instância do Chrome WebDriver com as definidas 'options' e 'service'
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [23]:

termo_busca = 'licitacao'

def coletar_todas_paginas():
    driver = iniciar_driver()
    url = f"https://www.gov.br/pf/pt-br/search?SearchableText={termo_busca}"
    driver.get(url)
    time.sleep(2)

    todas_noticias = []
    pagina = 1

    while True:
        print(f"\n🔎 Coletando dados da página {pagina}")

        # Pega os títulos
        noticias = driver.find_elements(By.CSS_SELECTOR, "span.titulo a")
        for noticia in noticias:
            titulo = noticia.text.strip()
            link = noticia.get_attribute("href")
            todas_noticias.append({"Título": titulo, "Link": link})
            print(f"✔ {titulo}")

        # Tenta localizar o botão "Próximo »"
        try:
            proximo = driver.find_element(By.XPATH, "//a[@class='proximo' and contains(text(), 'Próximo')]")
            time.sleep(1)
            driver.execute_script("arguments[0].scrollIntoView();", proximo)
            time.sleep(1)

            # Força o clique via JavaScript
            driver.execute_script("arguments[0].click();", proximo)

            pagina += 1
            time.sleep(2)
        except NoSuchElementException:
                print("🚫 Fim da paginação.")
                break

    driver.quit()
    return todas_noticias



In [ ]:
dados = coletar_todas_paginas()

In [ ]:

pd.DataFrame(dados).to_csv("noticias_licitacao_pf.csv", index=False, encoding="utf-8")
print(f"\n✅ Total de notícias coletadas: {len(dados)}")


✅ Total de notícias coletadas: 830


### Extração dos textos dentro da notícia

In [24]:
caminho_arquivo = 'noticias_licitacao_pf.csv'

dados_arq = pd.read_csv(caminho_arquivo, )

print(dados_arq['Link'])


0      https://www.gov.br/pf/pt-br/assuntos/noticias/...
1      https://www.gov.br/pf/pt-br/assuntos/noticias/...
2      https://www.gov.br/pf/pt-br/assuntos/noticias/...
3      https://www.gov.br/pf/pt-br/assuntos/noticias/...
4      https://www.gov.br/pf/pt-br/assuntos/noticias/...
                             ...                        
825    https://www.gov.br/pf/pt-br/assuntos/noticias/...
826    https://www.gov.br/pf/pt-br/assuntos/noticias/...
827    https://www.gov.br/pf/pt-br/assuntos/noticias/...
828    https://www.gov.br/pf/pt-br/assuntos/noticias/...
829    https://www.gov.br/pf/pt-br/assuntos/noticias/...
Name: Link, Length: 830, dtype: object


In [ ]:
def extrair_descricao_noticia():
    driver = iniciar_driver()
    partes_noticias = []

    #for i in range(0 , len(dados_arq['Link'])):
    for i in range(0 , 3):
        url = dados_arq['Link'][i]
        driver.get(url)
        time.sleep(0.5)
        descricao = driver.find_element(By.CLASS_NAME, "documentDescription").text.strip()
        partes_noticias.append({"Descrição": descricao, 'Link': url})

    driver.quit()
    return partes_noticias

In [ ]:
texto = extrair_descricao_noticia()
print(texto)

[{'Descrição': 'Operação objetiva reprimir a atuação de uma organização criminosa especializada na prática de fraudes em licitações públicas promovidas por diversos municípios do Estado e lavagem de dinheiro', 'Link': 'https://www.gov.br/pf/pt-br/assuntos/noticias/2025/06/pf-e-cgu-deflagram-operacao-de-combate-a-crimes-contra-a-administracao-publica-e-de-lavagem-de-capitais-em-pernambuco'}, {'Descrição': 'Três mandados de busca e apreensão são cumpridos por suspeitas de desvio de recursos públicos, provenientes da saúde e da educação, em Francinópolis/PI', 'Link': 'https://www.gov.br/pf/pt-br/assuntos/noticias/2025/06/pf-desarticula-esquema-criminoso-de-desvio-de-verbas-da-saude-e-da-educacao-no-interior-do-piaui'}, {'Descrição': 'O foragido possuía mandado de prisão preventiva expedido pela Justiça da Argentina em abril deste ano por crimes cometidos no final de 2024', 'Link': 'https://www.gov.br/pf/pt-br/assuntos/noticias/2025/05/pf-e-pmsc-prendem-cidadao-uruguaio-procurado-pela-inte

In [41]:
def extrair_texto_principal_noticia():
    driver = iniciar_driver()
    texto_principal = []

    #for i in range(0 , len(dados_arq['Link'])):
    for i in range(0 , 1):
        url = dados_arq['Link'][i]
        driver.get(url)
        time.sleep(0.5)
        descricao = driver.find_element(By.ID, "content-core").text.strip()
        data_publicacao = driver.find_element(By.CLASS_NAME, "value").text.strip()
        texto_principal.append({"Data publicação:": data_publicacao, "Texto Principal": descricao})

    driver.quit()
    return texto_principal

In [42]:
resultado = extrair_texto_principal_noticia()
print(resultado)

[{'Data publicação:': '05/06/2025 09h22', 'Texto Principal': 'Recife/PE. A Polícia Federal em Pernambuco, com apoio da Controladoria-Geral da União - CGU, deflagrou, na manhã de hoje (5/6), a Operação Firenze, com a finalidade de reprimir a atuação de uma organização criminosa especializada na prática de fraudes a licitações públicas promovidas por diversos municípios do Estado e lavagem de dinheiro.\nAs investigações, iniciadas em 2023, identificaram um suposto esquema criminoso de fraudes licitatórias consistente na frustração do caráter competitivo dos certames.\nApurou-se ainda que as empresas do grupo investigado mantêm contratos de terceirização de mão de obra com diversos municípios de Pernambuco, e com o governo estadual, as quais, somente entre os anos de 2021 e 2024, geraram receitas superiores à R$ 800 milhões para os envolvidos no esquema.\nPoliciais Federais e auditores da CGU deram cumprimento a 19 mandados de busca e apreensão nos municípios de Jaboatão dos Guararapes/PE